# 1. Preprocessing data:

### Import necessary libraries.

In [1]:
import pandas as pd
import numpy as np
import shapely
from shapely.geometry import shape, mapping
import geopandas as gpd
from geopandas.tools import sjoin
import requests 
import json
import matplotlib.pyplot as plt

/Users/gil/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
print('Necessary libraries have been imported successfully.')

Necessary libraries have been imported successfully.


Import csv files

In [3]:
tract2016 = pd.read_csv('2016Tract.csv')
tract2019 = pd.read_csv('2019Tract.csv')

In [4]:
tract2016['Geographic Area Name'] = tract2016['Geographic Area Name'].astype(str)
tract2016 = tract2016.rename(columns={'Geographic Area Name':"Tract"})


tract2019['Geographic Area Name'] = tract2019['Geographic Area Name'].astype(str)
tract2019 = tract2019.rename(columns={'Geographic Area Name':"Tract"})

In [5]:
processed2016 = tract2016[['City','Tract','Estimate!!SEX AND AGE!!Total population','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Cuban','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Other Hispanic or Latino','Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population']]
processed2019 = tract2019[['City','Tract','Estimate!!SEX AND AGE!!Total population','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Cuban','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Other Hispanic or Latino','Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population']]
processed2016 = processed2016.rename(columns={'Estimate!!SEX AND AGE!!Total population': 'Estimate Total Population','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican': 'Estimate Mexican Population', "Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican": "Estimate Puerto Rican Population","Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Cuban": "Estimate Cuban Population","Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Other Hispanic or Latino":"Estimate Other Hispanic or Latino","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population":"Estimate Voting Population: 18+"})
processed2019 = processed2019.rename(columns={'Estimate!!SEX AND AGE!!Total population': 'Estimate Total Population','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican': 'Estimate Mexican Population', "Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican": "Estimate Puerto Rican Population","Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Cuban": "Estimate Cuban Population","Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Other Hispanic or Latino":"Estimate Other Hispanic or Latino","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population":"Estimate Voting Population: 18+"})

In [6]:
processed2016

,City,Tract,Estimate Total Population,Estimate Mexican Population,Estimate Puerto Rican Population,Estimate Cuban Population,Estimate Other Hispanic or Latino,Estimate Voting Population: 18+
0,Acushnet,6541,6858,0,9,0,0,5525
1,Acushnet,6542,3556,0,137,0,20,2843
2,Lynn,2051,4893,0,133,33,1362,3236
3,Lynn,2052,5999,25,288,0,1500,3890
4,Lynn,2053,4206,0,112,0,290,3226
...,...,...,...,...,...,...,...,...
77,Springfield,8026,7418,0,2421,33,620,5001
78,Springfield,8026,1896,0,85,0,20,1559
79,Southwick,8131,5950,0,1,0,0,4645
80,Southwick,8131,3700,14,0,0,0,2919


In [7]:
pres2016 = pd.read_csv('2016PresidentPrecinct.csv')
pres2020 = pd.read_csv('2020PresidentPrecint.csv')
gov2018 = pd.read_csv('2018gov.csv')

In [8]:
pres2016.head()

,City/Town,Ward,Pct,Clinton/ Kaine,Trump/ Pence,Johnson/ Weld,Stein/ Baraka,Mcmullin/ Johnson,Kotlikoff/ Leamer,Feegbeh/ O'Brien,Moorehead/ Lilly,Schoenke/ Mitchel,All Others,No Preference,Blanks,Total Votes Cast
0,NaN,NaN,NaN,Democratic,Republican,Libertarian,Green-rainbow,Unenrolled,Unenrolled,Unenrolled,Unenrolled,Unenrolled,NaN,NaN,NaN,NaN
1,Abington,-,1,818,717,97,25,0,0,0,0,0,30,0.0,22,"1,709"
2,Abington,-,2,739,785,67,27,0,0,0,0,0,12,0.0,41,"1,671"
3,Abington,-,3,773,808,80,18,0,0,0,0,0,13,0.0,34,"1,726"
4,Abington,-,4,877,878,87,16,0,0,0,0,0,26,0.0,35,"1,919"


In [9]:
pres2016 = pres2016.drop(0,axis = 0)
pres2020 = pres2020.drop(0,axis = 0)
gov2018 = gov2018.drop(0,axis = 0)

In [10]:
pres2020.columns

Index(['City/Town', 'Ward', 'Precinct', 'Democratic', 'Republican',
       ' Libertarian ', 'Green-Rainbow', 'All Others', 'No Preference',
       'Blanks', 'Total Votes Cast', 'AREA_SQMI', 'AREA_ACRES', 'SHAPE_AREA',
       'SHAPE_LEN', 'geometry'],
      dtype='object')

In [11]:
pres2016 = pres2016.rename(columns = {'Clinton/ Kaine':'Democratic','Trump/ Pence':'Republican'})
pres2016 = pres2016[['City/Town','Ward','Pct','Democratic','Republican','Total Votes Cast']]
pres2020 = pres2020.rename(columns = {'Precinct':'Pct'})
pres2020 = pres2020[['City/Town','Ward','Pct','Democratic','Republican','Total Votes Cast']]
gov2018 = gov2018.rename(columns = {'Baker and Polito':'Republican','Gonzalez and Palfrey':'Democratic'})
gov2018 = gov2018[['City/Town','Ward','Pct','Democratic','Republican','Total Votes Cast']]

In [12]:
type(pres2016)

pandas.core.frame.DataFrame

In [13]:
pres2016.loc[pres2016['Ward']=='-','Ward'] = 'None'
pres2020.loc[pres2020['Ward']=='-','Ward'] = 'None'
gov2018.loc[gov2018['Ward']=='-','Ward'] = 'None'

In [14]:
pres2016

,City/Town,Ward,Pct,Democratic,Republican,Total Votes Cast
1,Abington,None,1,818,717,"1,709"
2,Abington,None,2,739,785,"1,671"
3,Abington,None,3,773,808,"1,726"
4,Abington,None,4,877,878,"1,919"
5,Abington,None,5,908,829,"1,887"
...,...,...,...,...,...,...
2171,Yarmouth,None,4,"1,002",860,"2,038"
2172,Yarmouth,None,5,881,797,"1,805"
2173,Yarmouth,None,6,907,753,"1,804"
2174,Yarmouth,None,7,"1,251","1,009","2,463"


In [15]:
pres2016['City/Town']=pres2016['City/Town'].str.upper()
pres2020['City/Town']=pres2020['City/Town'].str.upper()
gov2018['City/Town']=gov2018['City/Town'].str.upper()

### Get Latitude and Longitude of Each City using geopy and nominatim

In [16]:
from geopy.geocoders import Nominatim

In [18]:
pres2016['Latitude'] = -40.266666
pres2016['Longitude'] = 72.3452

pres2020['Latitude'] = -40.266666
pres2020['Longitude'] = 72.3452

gov2018['Latitude'] = -40.266666
gov2018['Longitude'] = 72.3452

In [19]:
pres2016

,City/Town,Ward,Pct,Democratic,Republican,Total Votes Cast,Latitude,Longitude
1,ABINGTON,None,1,818,717,"1,709",-40.266666,72.3452
2,ABINGTON,None,2,739,785,"1,671",-40.266666,72.3452
3,ABINGTON,None,3,773,808,"1,726",-40.266666,72.3452
4,ABINGTON,None,4,877,878,"1,919",-40.266666,72.3452
5,ABINGTON,None,5,908,829,"1,887",-40.266666,72.3452
...,...,...,...,...,...,...,...,...
2171,YARMOUTH,None,4,"1,002",860,"2,038",-40.266666,72.3452
2172,YARMOUTH,None,5,881,797,"1,805",-40.266666,72.3452
2173,YARMOUTH,None,6,907,753,"1,804",-40.266666,72.3452
2174,YARMOUTH,None,7,"1,251","1,009","2,463",-40.266666,72.3452


In [21]:
pres2016G = gpd.GeoDataFrame(
    pres2016, geometry=gpd.points_from_xy(pres2016.Longitude, pres2016.Latitude))

pres2020G = gpd.GeoDataFrame(
    pres2020, geometry=gpd.points_from_xy(pres2020.Longitude, pres2020.Latitude))

gov2018G = gpd.GeoDataFrame(
    gov2018, geometry=gpd.points_from_xy(gov2018.Longitude, gov2018.Latitude))

In [22]:
pres2016G

,City/Town,Ward,Pct,Democratic,Republican,Total Votes Cast,Latitude,Longitude,geometry
1,ABINGTON,None,1,818,717,"1,709",-40.266666,72.3452,POINT (72.34520 -40.26667)
2,ABINGTON,None,2,739,785,"1,671",-40.266666,72.3452,POINT (72.34520 -40.26667)
3,ABINGTON,None,3,773,808,"1,726",-40.266666,72.3452,POINT (72.34520 -40.26667)
4,ABINGTON,None,4,877,878,"1,919",-40.266666,72.3452,POINT (72.34520 -40.26667)
5,ABINGTON,None,5,908,829,"1,887",-40.266666,72.3452,POINT (72.34520 -40.26667)
...,...,...,...,...,...,...,...,...,...
2171,YARMOUTH,None,4,"1,002",860,"2,038",-40.266666,72.3452,POINT (72.34520 -40.26667)
2172,YARMOUTH,None,5,881,797,"1,805",-40.266666,72.3452,POINT (72.34520 -40.26667)
2173,YARMOUTH,None,6,907,753,"1,804",-40.266666,72.3452,POINT (72.34520 -40.26667)
2174,YARMOUTH,None,7,"1,251","1,009","2,463",-40.266666,72.3452,POINT (72.34520 -40.26667)


In [23]:
type(pres2016G)

geopandas.geodataframe.GeoDataFrame

In [24]:
pres2016G = pres2016G.rename(columns = {'Ward':'WARD','Pct':'PRECINCT','City/Town':'TOWN'})

pres2020G = pres2020G.rename(columns = {'Ward':'WARD','Pct':'PRECINCT','City/Town':'TOWN'})

gov2018G = gov2018G.rename(columns = {'Ward':'WARD','Pct':'PRECINCT','City/Town':'TOWN'})

In [25]:
pres2016G

,TOWN,WARD,PRECINCT,Democratic,Republican,Total Votes Cast,Latitude,Longitude,geometry
1,ABINGTON,None,1,818,717,"1,709",-40.266666,72.3452,POINT (72.34520 -40.26667)
2,ABINGTON,None,2,739,785,"1,671",-40.266666,72.3452,POINT (72.34520 -40.26667)
3,ABINGTON,None,3,773,808,"1,726",-40.266666,72.3452,POINT (72.34520 -40.26667)
4,ABINGTON,None,4,877,878,"1,919",-40.266666,72.3452,POINT (72.34520 -40.26667)
5,ABINGTON,None,5,908,829,"1,887",-40.266666,72.3452,POINT (72.34520 -40.26667)
...,...,...,...,...,...,...,...,...,...
2171,YARMOUTH,None,4,"1,002",860,"2,038",-40.266666,72.3452,POINT (72.34520 -40.26667)
2172,YARMOUTH,None,5,881,797,"1,805",-40.266666,72.3452,POINT (72.34520 -40.26667)
2173,YARMOUTH,None,6,907,753,"1,804",-40.266666,72.3452,POINT (72.34520 -40.26667)
2174,YARMOUTH,None,7,"1,251","1,009","2,463",-40.266666,72.3452,POINT (72.34520 -40.26667)


### Read in shapefiles

In [26]:
MA_p = gpd.read_file("wardsprecincts_poly/WARDSPRECINCTS_POLY.shp")

In [27]:
MA_t = gpd.read_file("CENSUS2010_BLK_BG_TRCT_SHP/CENSUS2010TRACTS_POLY.shp")

In [30]:
MA_t.columns

Index(['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10',
       'NAMELSAD10', 'MTFCC10', 'ALAND10', 'AWATER10', 'INTPTLAT10',
       'INTPTLON10', 'AREA_SQFT', 'AREA_ACRES', 'POP100_RE', 'HU100_RE',
       'LOGPL94171', 'LOGSF1', 'LOGACS0610', 'LOGSF1C', 'SHAPE_AREA',
       'SHAPE_LEN', 'geometry'],
      dtype='object')

In [28]:
type(MA_p)

geopandas.geodataframe.GeoDataFrame

In [32]:
MA_t.head()

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,ALAND10,AWATER10,INTPTLAT10,...,AREA_ACRES,POP100_RE,HU100_RE,LOGPL94171,LOGSF1,LOGACS0610,LOGSF1C,SHAPE_AREA,SHAPE_LEN,geometry
0,25,021,418003,25021418003,4180.03,Census Tract 4180.03,G5020,1705668.0,2936.0,+42.2350240,...,422.1740,2481,1350,0141872,123764,0003690,0123764,1.708484e+06,6274.185034,"POLYGON ((240678.759 886748.073, 240607.995 88..."
1,25,021,417701,25021417701,4177.01,Census Tract 4177.01,G5020,1543651.0,12275.0,+42.2523398,...,384.4502,5417,2983,0141838,123730,0003683,0123730,1.555821e+06,6229.137913,"POLYGON ((240969.666 890122.806, 240985.156 89..."
2,25,021,417702,25021417702,4177.02,Census Tract 4177.02,G5020,1685529.0,317947.0,+42.2582818,...,416.4918,2765,1109,0141843,123735,0003684,0123735,1.685489e+06,8542.498286,"POLYGON ((242328.221 890545.050, 242276.888 89..."
3,25,021,418102,25021418102,4181.02,Census Tract 4181.02,G5020,771203.0,2371.0,+42.2470410,...,191.1397,3377,1676,0141882,123774,0003693,0123774,7.735182e+05,4540.296101,"POLYGON ((240372.666 888331.030, 240393.057 88..."
4,25,021,418004,25021418004,4180.04,Census Tract 4180.04,G5020,1316466.0,0.0,+42.2383744,...,325.2812,4280,1987,0141874,123766,0003691,0123766,1.316372e+06,5854.497366,"POLYGON ((240687.299 888298.408, 240700.449 88..."


In [34]:
jointest = gpd.sjoin(MA_t, MA_p, how="left", op="intersects")

In [47]:
jointest[20:60]

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,ALAND10,AWATER10,INTPTLAT10,...,PRECINCT,DISTRICT,POP_2010,TOWN,TOWN_ID,AREA_SQMI,AREA_ACRES_right,YEAR,SHAPE_AREA_right,SHAPE_LEN_right
3,25,021,418102,25021418102,4181.02,Census Tract 4181.02,G5020,771203.0,2371.0,+42.2470410,...,4,4-4,3187,QUINCY,243,0.260,166.38,2012,6.733051e+05,4259.449904
3,25,021,418102,25021418102,4181.02,Census Tract 4181.02,G5020,771203.0,2371.0,+42.2470410,...,1,5-1,3051,QUINCY,243,0.376,240.66,2012,9.739163e+05,7059.653936
3,25,021,418102,25021418102,4181.02,Census Tract 4181.02,G5020,771203.0,2371.0,+42.2470410,...,1,3-1,3192,QUINCY,243,0.488,312.28,2012,1.263774e+06,7696.171565
3,25,021,418102,25021418102,4181.02,Census Tract 4181.02,G5020,771203.0,2371.0,+42.2470410,...,2,4-2,3226,QUINCY,243,0.378,242.14,2012,9.799091e+05,5038.177806
4,25,021,418004,25021418004,4180.04,Census Tract 4180.04,G5020,1316466.0,0.0,+42.2383744,...,1,4-1,3226,QUINCY,243,4.766,3050.52,2012,1.234507e+07,21388.791139
4,25,021,418004,25021418004,4180.04,Census Tract 4180.04,G5020,1316466.0,0.0,+42.2383744,...,2B,2B,2935,BRAINTREE,40,0.692,442.80,2012,1.791962e+06,6620.210939
4,25,021,418004,25021418004,4180.04,Census Tract 4180.04,G5020,1316466.0,0.0,+42.2383744,...,5,4-5,3143,QUINCY,243,0.434,277.72,2012,1.123883e+06,4941.645638
4,25,021,418004,25021418004,4180.04,Census Tract 4180.04,G5020,1316466.0,0.0,+42.2383744,...,5,2-5,2957,QUINCY,243,0.504,322.57,2012,1.305400e+06,6952.381657
4,25,021,418004,25021418004,4180.04,Census Tract 4180.04,G5020,1316466.0,0.0,+42.2383744,...,4,4-4,3187,QUINCY,243,0.260,166.38,2012,6.733051e+05,4259.449904
4,25,021,418004,25021418004,4180.04,Census Tract 4180.04,G5020,1316466.0,0.0,+42.2383744,...,1,5-1,3051,QUINCY,243,0.376,240.66,2012,9.739163e+05,7059.653936


In [33]:
MA_p.head()

,WP_NAME,WARD,PRECINCT,DISTRICT,POP_2010,TOWN,TOWN_ID,AREA_SQMI,AREA_ACRES,YEAR,SHAPE_AREA,SHAPE_LEN,geometry
0,Braintree Town Precinct 5B,None,5B,5B,2883,BRAINTREE,40,1.217,778.80,2012,3.151713e+06,8711.937896,"POLYGON ((242364.517 883741.979, 242359.642 88..."
1,Braintree Town Precinct 6A,None,6A,6A,3070,BRAINTREE,40,1.994,1275.96,2012,5.163667e+06,11175.567690,"POLYGON ((243479.798 881994.897, 243477.538 88..."
2,Braintree Town Precinct 6B,None,6B,6B,2925,BRAINTREE,40,1.134,725.97,2012,2.937925e+06,9455.882837,"POLYGON ((241614.364 881551.551, 241602.936 88..."
3,Chelsea City Ward 1 Precinct 1,1,1,1-1,2295,CHELSEA,57,0.139,88.82,2012,3.594632e+05,3340.823153,"POLYGON ((239281.036 904133.287, 239277.513 90..."
4,Chelsea City Ward 1 Precinct 2,1,2,1-2,2108,CHELSEA,57,0.073,47.04,2012,1.903509e+05,2247.883591,"POLYGON ((238408.578 904480.138, 238393.140 90..."


In [37]:
#MA_p['WARD'] = MA_p['WARD'].where(type(MA_p['WARD'])!=type(None), 'None')
#MA_l = MA_p['WARD'].mask(lambda x : x == 'None', other = 'hi')
MA_p['WARD'] = MA_p['WARD'].astype(str)

In [38]:
#hi = (MA_p['WARD'][6])

In [39]:
#MA_p['WARD'][0].isna(inplace = True)

In [40]:
MA_p.head(10)

,WP_NAME,WARD,PRECINCT,DISTRICT,POP_2010,TOWN,TOWN_ID,AREA_SQMI,AREA_ACRES,YEAR,SHAPE_AREA,SHAPE_LEN,geometry
0,Braintree Town Precinct 5B,None,5B,5B,2883,BRAINTREE,40,1.217,778.80,2012,3.151713e+06,8711.937896,"POLYGON ((242364.517 883741.979, 242359.642 88..."
1,Braintree Town Precinct 6A,None,6A,6A,3070,BRAINTREE,40,1.994,1275.96,2012,5.163667e+06,11175.567690,"POLYGON ((243479.798 881994.897, 243477.538 88..."
2,Braintree Town Precinct 6B,None,6B,6B,2925,BRAINTREE,40,1.134,725.97,2012,2.937925e+06,9455.882837,"POLYGON ((241614.364 881551.551, 241602.936 88..."
3,Chelsea City Ward 1 Precinct 1,1,1,1-1,2295,CHELSEA,57,0.139,88.82,2012,3.594632e+05,3340.823153,"POLYGON ((239281.036 904133.287, 239277.513 90..."
4,Chelsea City Ward 1 Precinct 2,1,2,1-2,2108,CHELSEA,57,0.073,47.04,2012,1.903509e+05,2247.883591,"POLYGON ((238408.578 904480.138, 238393.140 90..."
5,Chelsea City Ward 1 Precinct 3,1,3,1-3,2268,CHELSEA,57,0.061,39.11,2012,1.582884e+05,1947.861606,"POLYGON ((239021.319 904577.600, 239001.380 90..."
6,Chelsea City Ward 1 Precinct 4,1,4,1-4,2110,CHELSEA,57,0.275,176.02,2012,7.123119e+05,6290.864945,"POLYGON ((239281.036 904133.287, 239275.121 90..."
7,East Bridgewater Town Precinct 1,None,1,1,3450,EAST BRIDGEWATER,83,4.306,2755.90,2012,1.115278e+07,13768.076237,"POLYGON ((245439.900 867976.931, 245464.150 86..."
8,East Bridgewater Town Precinct 3,None,3,3,3538,EAST BRIDGEWATER,83,5.051,3232.91,2012,1.308317e+07,16015.864863,"POLYGON ((248982.673 868095.257, 248983.707 86..."
9,East Bridgewater Town Precinct 4,None,4,4,3324,EAST BRIDGEWATER,83,4.067,2603.02,2012,1.053411e+07,14091.278987,"POLYGON ((247062.568 862095.141, 246965.634 86..."


In [41]:
#MA_p['WARD'] = MA_p['WARD'].gpd.replace('None', '-', inplace=True)

In [42]:
#pres2016.loc[type(MA_p['WARD'])=='NoneType','WARD'] = 'None'

In [43]:
join = gpd.pd.merge(MA_p, pres2016G,  how='left', left_on=['PRECINCT','WARD','TOWN'], right_on = ['PRECINCT','WARD','TOWN'])
#join = MA_p.merge(pres2016G, on = 'PRECINCT', how = 'left')

In [46]:
join.head()

,WP_NAME,WARD,PRECINCT,DISTRICT,POP_2010,TOWN,TOWN_ID,AREA_SQMI,AREA_ACRES,YEAR,SHAPE_AREA,SHAPE_LEN,geometry_x,Democratic,Republican,Total Votes Cast,Latitude,Longitude,geometry_y
0,Braintree Town Precinct 5B,None,5B,5B,2883,BRAINTREE,40,1.217,778.80,2012,3.151713e+06,8711.937896,"POLYGON ((242364.517 883741.979, 242359.642 88...",994,762,"1,928",-40.266666,72.3452,POINT (72.34520 -40.26667)
1,Braintree Town Precinct 6A,None,6A,6A,3070,BRAINTREE,40,1.994,1275.96,2012,5.163667e+06,11175.567690,"POLYGON ((243479.798 881994.897, 243477.538 88...",960,864,"1,984",-40.266666,72.3452,POINT (72.34520 -40.26667)
2,Braintree Town Precinct 6B,None,6B,6B,2925,BRAINTREE,40,1.134,725.97,2012,2.937925e+06,9455.882837,"POLYGON ((241614.364 881551.551, 241602.936 88...",889,785,"1,825",-40.266666,72.3452,POINT (72.34520 -40.26667)
3,Chelsea City Ward 1 Precinct 1,1,1,1-1,2295,CHELSEA,57,0.139,88.82,2012,3.594632e+05,3340.823153,"POLYGON ((239281.036 904133.287, 239277.513 90...",429,92,573,-40.266666,72.3452,POINT (72.34520 -40.26667)
4,Chelsea City Ward 1 Precinct 2,1,2,1-2,2108,CHELSEA,57,0.073,47.04,2012,1.903509e+05,2247.883591,"POLYGON ((238408.578 904480.138, 238393.140 90...",297,44,376,-40.266666,72.3452,POINT (72.34520 -40.26667)


In [ ]:
pres2020G

In [ ]:
ptest = pres2020G.dropna()

In [ ]:
ptest[9:50]

In [ ]:
join2020 = gpd.pd.merge(MA_p, pres2020G,  how='left', left_on=['PRECINCT','WARD','TOWN'], right_on = ['PRECINCT','WARD','TOWN'])

In [ ]:
#join2020=join2020.dropna()

In [ ]:
join2018 = gpd.pd.merge(MA_p, gov2018G,  how='left', left_on=['PRECINCT','WARD','TOWN'], right_on = ['PRECINCT','WARD','TOWN'])



In [ ]:
pres2016G

In [ ]:
join = join.drop(columns = {'Latitude','Longitude','geometry_y'})
join = join.rename(columns = {'geometry_x':'geometry'})

In [ ]:
join2020 = join2020.drop(columns = {'Latitude','Longitude','geometry_y'})
join2020 = join2020.rename(columns = {'geometry_x':'geometry'})

In [ ]:
join2020

In [ ]:
join2018

In [ ]:
join2018 = join2018.drop(columns = ['Latitude','Longitude','geometry_y'])
join2018 = join2018.rename(columns = {'geometry_x':'geometry'})

In [ ]:
join2018

In [ ]:
join.head()

In [ ]:
type(join)

In [ ]:
join = join.fillna('0')

#join2020 = join2020.fillna('0')

join2018 = join2018.fillna('0')

In [ ]:
join['Democratic'] = join['Democratic'].str.replace(',','').astype('float')
join['Republican'] = join['Republican'].str.replace(',','').astype('float')
join['Total Votes Cast'] = join['Total Votes Cast'].str.replace(',','').astype('float')

# join2020['Democratic'] = join2020['Democratic'].str.replace(',','').astype('float')
# join2020['Republican'] = join2020['Republican'].str.replace(',','').astype('float')
# join2020['Total Votes Cast'] = join2020['Total Votes Cast'].str.replace(',','').astype('float')

join2018['Democratic'] = join2018['Democratic'].str.replace(',','').astype('float')
join2018['Republican'] = join2018['Republican'].str.replace(',','').astype('float')
join2018['Total Votes Cast'] = join2018['Total Votes Cast'].str.replace(',','').astype('float')

In [ ]:
join2020.head()

In [ ]:
# join2020['Democratic'] = join2020['Democratic'].str.replace(',','').astype('float')
# join2020['Republican'] = join2020['Republican'].str.replace(',','').astype('float')
# join2020['Total Votes Cast'] = join2020['Total Votes Cast'].str.replace(',','').astype('float')



In [ ]:
join2020

In [ ]:
join.columns

In [ ]:
join['% Democratic'] = join['Democratic']/(join['Total Votes Cast'])
join['% Republican'] = join['Republican']/(join['Total Votes Cast'])

join2020['% Democratic'] = join2020['Democratic']/(join2020['Total Votes Cast'])
join2020['% Republican'] = join2020['Republican']/(join2020['Total Votes Cast'])

join2018['% Democratic'] = join2018['Democratic']/(join2018['Total Votes Cast'])
join2018['% Republican'] = join2018['Republican']/(join2018['Total Votes Cast'])

In [ ]:
jointest.head()

In [ ]:
jointests = pd.merge(jointest, processed2016,  how='left', left_on='NAME10', right_on ='Tract' )
#merge all the data together 
#map different level
#Send the mapping over to stephen bachman

In [ ]:
jointests.head()

In [ ]:
type(join)

In [ ]:
# join2020['% Democratic'] = join2020['Democratic']/(join2020['Total Votes Cast'])
# join2020['% Republican'] = join2020['Republican']/(join2020['Total Votes Cast'])



In [ ]:
join2020

In [ ]:
Dem2016high = join.loc[(join["% Democratic"] > .65)]
Dem2016low = join.loc[(join["% Democratic"] <= .65)&(join["% Democratic"] > .5)]
Rep2016high = join.loc[(join["% Republican"] > .65)]
Rep2016low = join.loc[(join["% Republican"] <= .65)&(join["% Republican"] > .5)]
nomaj = join.loc[(join["% Republican"] <=.5)&(join["% Democratic"] <=.5)]

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (20,10))
ax.set_aspect('equal')
join.plot(ax = ax, color = 'white', edgecolor = 'black')
Dem2016high.plot(ax=ax, color = 'blue', label = 'High Democratic Support')
Dem2016low.plot(ax=ax, color = 'lightblue', label = 'Low Democratic Support')
Rep2016high.plot(ax=ax, color = 'red', label = 'High Republican Support')
Rep2016low.plot(ax=ax, color = 'salmon', label = 'Low Republican Support')
nomaj.plot(ax=ax, color = 'grey', label = 'No High Majority')
plt.legend()
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
plt.title("Geospatial Analysis of 2016 Presidential Election")
plt.show()
#plt.savefig('PresidentMap2016a.png')

In [ ]:
join2020

In [ ]:
Dem2016high = join2020.loc[(join2020["% Democratic"] > .65)]
Dem2016low = join2020.loc[(join2020["% Democratic"] <= .65)&(join2020["% Democratic"] > .5)]
Rep2016high = join2020.loc[(join2020["% Republican"] > .65)]
Rep2016low = join2020.loc[(join2020["% Republican"] <= .65)&(join2020["% Republican"] > .5)]
nomaj = join2020.loc[(join2020["% Republican"] <=.5)&(join2020["% Democratic"] <=.5)]

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (20,10))
ax.set_aspect('equal')
join2020.plot(ax = ax, color = 'white', edgecolor = 'black')
Dem2016high.plot(ax=ax, color = 'blue', label = 'High Democratic Support')
Dem2016low.plot(ax=ax, color = 'lightblue', label = 'Low Democratic Support')
Rep2016high.plot(ax=ax, color = 'red', label = 'High Republican Support')
Rep2016low.plot(ax=ax, color = 'salmon', label = 'Low Republican Support')
nomaj.plot(ax=ax, color = 'grey', label = 'No High Majority')
plt.legend(loc = 'lower left')
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
plt.title("Geospatial Analysis of 2020 Presidential Election")
plt.show()
#plt.savefig('PresidentMap2016a.png')

In [ ]:
join2020

In [ ]:
Dem2018high = join2018.loc[(join2018["% Democratic"] > .65)]
Dem2018low = join2018.loc[(join2018["% Democratic"] <= .65) & (join2018["% Democratic"] > .5)]
Rep2018high = join2018.loc[(join2018["% Republican"] > .65)]
Rep2018low = join2018.loc[(join2018["% Republican"] <= .65) & (join2018["% Republican"] > .5)]
nomaj = join2018.loc[(join2018["% Republican"] <=.5) & (join2018["% Democratic"] <=.5)]

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (20,10))
ax.set_aspect('equal')
join2018.plot(ax = ax, color = 'white', edgecolor = 'black')
Dem2018high.plot(ax=ax, color = 'blue', label = 'High Democratic Support')
Dem2018low.plot(ax=ax, color = 'lightblue', label = 'Low Democratic Support')
Rep2018high.plot(ax=ax, color = 'red', label = 'High Republican Support')
Rep2018low.plot(ax=ax, color = 'salmon', label = 'Low Republican Support')
nomaj.plot(ax=ax, color = 'grey', label = 'No High Majority')
plt.legend(loc = 'lower left')
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
plt.title("Geospatial Analysis of 2018 Governor Election")
plt.show()
#plt.savefig('PresidentMap2016a.png')
#

In [ ]:
join2018